In [1]:
import os
os.environ['SPARK_HOME']='/home/envmodules/lib/spark-2.2.0-bin-hadoop2.7/'
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType

spark = SparkSession.builder.appName('aaerLabelCreation').getOrCreate()

annual_compustat = spark.read.csv('/user/vcs/annual_compustat.csv', header=True, inferSchema=True)
dgls = spark.read.csv('/user/vcs/dgls_integrated.csv', header=True, inferSchema=True)
ibes_complete = spark.read.csv('/user/vcs/ibes.csv', header=True, inferSchema=True)

ls = ['CONM', 'ACCHG', 'ACO', 'ACOMINC', 'ACQCSHI', 'ACQGDWL', 'ACQINTAN', 'ACQLNTAL', 'ACT', 'AMGW','AQ','AP','AQC','ARC','AT','AU','AUOP','AUOPIC','BKVLPS','CAPX','CDVC','CEQ','CGA','CHE','CHECH','CI','CIMII','CLD2','CLD3','CLD4','CLD5','CLT','COGS','CSHI','CURNCD','DCOM','DCPSTK','DCVT','DEPC','DLTT','DM','DO','DP','DPACLS','DRC','DRLT','DV','DVC','DVPA','DVPDP','DVPSP_F','EBIT','EBITDA','EMOL','EMP','EPSFI','EPSPI','ESOPCT','ESOPDLT','EXCHG','EXRE','FATB','FATE','FATN','FATL','FATO','FCA','FEL','FIC','FINCF','FOPT','FYEAR','FYR','GDWL','GDWLAM','GDWLIA','GLA','GP','GWO','HEDGEGL','ICAPT','IDBFLAG','IDIIS','IDILC','IDIT','INTAN','INTC','INVCH','INCFG','INVOFS','INVRM','INVT','INVWIP','IPODATE','ISEQ','ISEQC','ISEQM','ISFI','ISGR','ISGU','ITCB','IVCH','IVNCF','LCT','LCUACU','LIFR','LLRCI','LLWOCI','LT','MIB','MIBT','MII','MKVALT','NAICS','NI','NIINT','NIINTPFC','NIINTPFP','NOPIO','NP','NPANL','NPAT','NRTXT','OANCF','OB','OIADP','OIBDP','OPEPS','OPINI','OPITI','OPREPSX','OPTCA','OPTEX','OPTEXD','OPTFVGR','OPTGR','OPTPRCCA','OPTPRCEX','OPTPRCEY','OPTPRCGR','OPTPRCWA','OPTVOL','PCL','PI','PIFO','PLL','PNCA','PNCIA','PPEGT','PPENB','PPENC','PPENLS','PPENNR','PPENT','PPEVO','PPEVR','PRCC_F','PRCH_F','PRICAN','PRIROW','PRIUSA','PRODV','PRSTKCC','PRSTKPC','PSTKC','PVCL','PVO','PVPL','PVT','RANK','RCA','RDIP','RE','REA','REAJO','RECCH','RECT','RECTA','REUNA','REVT','RLL','RMUM','RPAG','RVLRV','RVTI','SALE','SALEPFC','SALEPFP','SCO','SCSTKC','SEQ','SIC','SIV','SPCE','SPCINDCD','SPCSECCD','SPCSRC','SPI','SPPE','SPPIV','SPSTKC','SRET','SSNP','STBO','STIO','STKCO','STKO','TDC','TEQ','TFVA','TFVCE','TFVL','TIC','TLCF','TRANSA','TSA','TSTKC','TSTKP','TXC','TXDB','TXDBA','TXDBCA','TXDBCL','TXFED','TXFO','TXPD','TXR','TXT','TXTUBXINTBS','TXTUBXINTIS','VPAC','VPO','WCAP','WCAPC','WCAPCH','WDA','XACC','XAD','XAGO','XAGT','XCOM','XDP','XEQO','XI','XIDO','XINST','XINT','XINTD','XINTOPT','XLR','XOPR','XOPRAR','XPP','XPR','XRD','XRENT','XS','XSGA','XSTF','XSTFO','XSTFWS','XT']

ls = [x.lower() for x in ls]

ls.append('eps_avg_ibes')

annual_filtered = annual_compustat.select([c for c in annual_compustat.columns if c in ls])

cond = [dgls['YEARA'] == annual_filtered['fyear'], dgls['COMPUSTAT IDENTIFIER'] == annual_filtered['conm']]

annual_joined = annual_filtered.join(dgls, cond,'left')

In [2]:
annual_compustat.count()

505781

In [3]:
annual_joined.count()

505848

In [4]:
ibes_complete = ibes_complete.withColumn('ibes_filing_month_day', ibes_complete.FPEDATS[5:8])

In [5]:
ibes_complete.filter(ibes_complete.ibes_filing_month_day == '1231').count()

1163195

In [6]:
ibes_complete.show()

+------+-----+--------+---------+---+-------+-----+--------+--------+--------+-----------+-----------+---------------------+
|TICKER|OFTIC| ACTDATS|ESTIMATOR|FPI|MEASURE|VALUE| FPEDATS| ACTTIMS| ANNDATS|ANNDATS_ACT|ANNTIMS_ACT|ibes_filing_month_day|
+------+-----+--------+---------+---+-------+-----+--------+--------+--------+-----------+-----------+---------------------+
|  0000| TLMR|20140311|      149|  1|    EPS| 0.73|20141231|15:17:12|20140309|   20150130|   16:30:00|                 1231|
|  0000| TLMR|20140311|      873|  1|    EPS|  0.8|20141231|17:10:50|20140310|   20150130|   16:30:00|                 1231|
|  0000| TLMR|20140311|      228|  1|    EPS| 0.83|20141231|15:49:22|20140310|   20150130|   16:30:00|                 1231|
|  0000| TLMR|20140311|      952|  1|    EPS| 0.66|20141231|15:22:08|20140310|   20150130|   16:30:00|                 1231|
|  0000| TLMR|20140317|     1267|  1|    EPS| 0.86|20141231|20:13:35|20140317|   20150130|   16:30:00|                 1231|


In [7]:
test = annual_joined.join(ibes_complete, ibes_complete['OFTIC'] == annual_joined['tic'], how='left')

In [8]:
test.count()

38679346

In [9]:
ibes_complete.count()

1455831

In [10]:
ibes_complete = ibes_complete.withColumn('ibes_filing_year', ibes_complete.FPEDATS[1:4])

In [11]:
ibes_complete.first()

Row(TICKER='0000', OFTIC='TLMR', ACTDATS=20140311, ESTIMATOR=149, FPI=1, MEASURE='EPS', VALUE=0.73, FPEDATS=20141231, ACTTIMS='15:17:12', ANNDATS=20140309, ANNDATS_ACT=20150130, ANNTIMS_ACT='16:30:00', ibes_filing_month_day='1231', ibes_filing_year='2014')

In [12]:
ibes_complete = ibes_complete.filter(ibes_complete.MEASURE=='EPS')

In [13]:
ibes_complete.createOrReplaceTempView("ibes")
sqlDF = spark.sql("SELECT OFTIC, ibes_filing_year, AVG(VALUE) as eps_avg_ibes FROM ibes GROUP BY OFTIC, ibes_filing_year") #GROUP BY OFTIC ORDER BY FPEDATS DESC LIMIT 1")
sqlDF.show()
ibes_complete = sqlDF

+-----+----------------+--------------------+
|OFTIC|ibes_filing_year|        eps_avg_ibes|
+-----+----------------+--------------------+
| PAYC|            2016|  0.7163043478260869|
| PARN|            2016| -1.6555399999999998|
| CHRS|            2016| -4.6560476190476185|
| VTGN|            2017| -1.8016666666666667|
| TIER|            2016|               -0.63|
| MTCH|            2016|  0.7208450704225354|
|  FBK|            2016|  2.1169230769230767|
| BMRA|            2016|-0.04999999999999...|
| BSTC|            2017|              1.6575|
| AMED|            2015|  1.3168372093023253|
| AAON|            2015|  0.8535714285714286|
|ASBFY|            2012|                1.39|
| NGNM|            2012|0.009999999999999998|
|  ACO|            2011|  1.9080769230769237|
| ATEC|            2011|0.013846153846153854|
|ACXIF|            2016|   5.400448979591838|
|  ADC|            2016|              1.6712|
| ADNC|            2014| -0.6402000000000001|
|  AEG|            2017|  0.707691

In [14]:
final_joined = annual_joined.join(ibes_complete, on = [ibes_complete['OFTIC'] == annual_joined['tic'], ibes_complete['ibes_filing_year'] == annual_joined['fyear']], how = 'left_outer')

In [15]:
final_joined.columns

['fyear',
 'tic',
 'conm',
 'curncd',
 'fyr',
 'acchg',
 'aco',
 'acominc',
 'acqcshi',
 'acqgdwl',
 'acqintan',
 'acqlntal',
 'act',
 'amgw',
 'ap',
 'aqc',
 'arc',
 'at',
 'bkvlps',
 'capx',
 'cdvc',
 'ceq',
 'cga',
 'che',
 'chech',
 'ci',
 'cimii',
 'cld2',
 'cld3',
 'cld4',
 'cld5',
 'clt',
 'cogs',
 'cshi',
 'dcom',
 'dcpstk',
 'dcvt',
 'depc',
 'dltt',
 'dm',
 'do',
 'dp',
 'dpacls',
 'drc',
 'drlt',
 'dv',
 'dvc',
 'dvpa',
 'dvpdp',
 'ebit',
 'ebitda',
 'emol',
 'emp',
 'epsfi',
 'epspi',
 'esopct',
 'esopdlt',
 'exre',
 'fatb',
 'fate',
 'fatl',
 'fatn',
 'fato',
 'fca',
 'fel',
 'fincf',
 'fopt',
 'gdwl',
 'gdwlam',
 'gdwlia',
 'gla',
 'gp',
 'gwo',
 'hedgegl',
 'icapt',
 'idiis',
 'idilc',
 'idit',
 'intan',
 'intc',
 'invch',
 'invofs',
 'invrm',
 'invt',
 'invwip',
 'iseq',
 'iseqc',
 'iseqm',
 'isfi',
 'isgr',
 'isgu',
 'itcb',
 'ivch',
 'ivncf',
 'lct',
 'lcuacu',
 'lifr',
 'llrci',
 'llwoci',
 'lt',
 'mib',
 'mibt',
 'mii',
 'ni',
 'niint',
 'niintpfc',
 'niintpfp',
 'n

In [16]:
final_joined.show()

+-----+-----+--------------------+------+---+-----+------+-------+-------+-------+--------+--------+-------+----+--------+-----+----+--------+-------+-------+-------+--------+----+------+------+------+-----+------+------+------+------+----+--------+-------+----+------+------+-------+--------+--------+----+-------+------+----+----+-------+-------+----+-----+-------+-------+----+------+-----+-----+------+-------+------+-----+------+-----+----+----+------+----+--------+-------+------+------+------+----+-------+----+-------+--------+-----+-----+------+------+-----+------+------+-----+-------+-------+----+-----+-----+----+----+----+-------+------+--------+-------+--------+----+-----+------+--------+-----+-----+----+-------+------+--------+--------+------+-----+-----+-----+-----+-------+------+-------+-------+-----+-----+-----+-------+-----+------+------+-------+-----+--------+--------+--------+--------+--------+------+----+-------+-----+----+----+-----+--------+-----+-------+------+------+-

In [17]:
def createFlag(yeara):
	if yeara is None:
		return 0.0
	else:
		return 1.0

createLabelFlag = udf(createFlag, DoubleType())

# Creating a flag field which is equal to 1.0 when there is matching record in DGLS otherwise 0.0
final_joined = final_joined.withColumn('label', createLabelFlag(final_joined.YEARA))

ls.append('label')
final_joined = final_joined.drop('FYR').drop('COGS')
final_to_keep = final_joined.select([c for c in final_joined.columns if c in ls]).cache()


# Writing the joinned csv / parquet to hdfs
# try:
#     final_to_keep.coalesce(1).write.csv('/user/vcs/integrated_dataset_with_labels')
# except:
#     pass

# try:
#     final_to_keep.write.parquet('annual_integrated_dataset_with_labels.parquet')
# except:
#     pass

In [18]:
final_to_keep.show()

+-----+-----+--------------------+------+-------+-----+-------+-------+-------+--------+--------+------+----+-------+-----+----+---------+--------+------+----+--------+----+--------+-------+----+-----+-----+----+----+----+----+-------+----+------+-----+----+-------+-------+------+------+------+----+----+-----+------+-----+-----+-------+-------+----+-----+-----+-----+------+-------+-----+-----+-------+------+----+----+-----+----+--------+------+------+------+------+----+-------+----+-------+--------+-----+-------+-------+-------+----+------+------+------+------+------+----+-----+-----+----+----+----+----+-----+-------+------+---------+-----+-----+------+---------+-----+-----+------+-------+-----+--------+--------+------+--------+-----+----+-----+-------+------+-------+-------+-----+-----+-----+-------+-----+-----+------+-------+-----+--------+--------+--------+--------+--------+------+----+-------+-----+----+------+-----+-------+------+-----+------+------+-------+------+-----+-----+----

In [19]:
annual_compustat.count()

505781

In [20]:
final_to_keep.count()

505848

In [21]:
final_to_keep.select('fyear', 'eps_avg_ibes')

DataFrame[fyear: int, eps_avg_ibes: double]

In [22]:
test_df = spark.read.parquet('/user/vcs/annual_integrated_dataset_v2.parquet')

In [23]:
test_df.count()

46045046

In [ ]:
# Writing the joinned csv / parquet to hdfs
try:
    final_to_keep.coalesce(1).write.csv('/user/vcs/integrated_dataset_with_labels_ibes_fix_v2')
except:
    pass

In [ ]:
try:
    final_to_keep.write.parquet('/user/vcs/annual_integrated_dataset_with_labels_ibes_fix_v2.parquet')
except:
    pass